# 2022-72 SANPOPSIM Outputs QC - DLE

- [QC Test 8: School age population](#school)
- QC Test 12: Class of Worker vs. EDD
- [QC Test 22: Ratio of workers to employment](#workers_ratio)

In [2]:
import pandas as pd
import numpy as np
import pyodbc
from sodapy import Socrata

## Data Preparation

In [35]:
synthetic_households = pd.read_csv('C:/Users/dle/OneDrive - San Diego Association of Governments/Projects/2022/2022-72 SANDPOPSIM Output QC/Population Sim Outputs/synthetic_households.csv')
synthetic_persons = pd.read_csv('C:/Users/dle/OneDrive - San Diego Association of Governments/Projects/2022/2022-72 SANDPOPSIM Output QC/Population Sim Outputs/synthetic_persons.csv')

In [3]:
synthetic_households.head()

,household_id,tract,mgra,NP,HHADJINC,ADJINC,HHT,WIF,HUPAC,VEH,numWorkers,GQ_type
0,1,18100.0,245,3,121500.0,1010145,3.0,2.0,4.0,3.0,1.0,NaN
1,2,18100.0,245,3,75400.0,1010145,1.0,2.0,1.0,2.0,2.0,NaN
2,3,18100.0,245,3,45800.0,1010145,3.0,1.0,4.0,1.0,1.0,NaN
3,4,18100.0,245,3,70000.0,1010145,1.0,2.0,1.0,2.0,2.0,NaN
4,5,18100.0,245,1,35700.0,1010145,6.0,NaN,4.0,1.0,1.0,NaN


In [51]:
synthetic_persons.head()

,tract,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,isWorker
0,18100.0,245,1,1,52,2,1.0,1.0,40.0,NaN,1,2,4.0,1
1,18100.0,245,1,2,39,1,3.0,1.0,40.0,NaN,1,2,4.0,0
2,18100.0,245,1,3,79,2,6.0,NaN,NaN,NaN,1,2,4.0,0
3,18100.0,245,2,1,25,2,4.0,5.0,50.0,15.0,6,2,1.0,1
4,18100.0,245,2,2,24,1,1.0,6.0,20.0,15.0,1,1,2.0,1


<a id='school'></a>
## QC Test 8: Examine distribution of school age population

Compare SANPOPSIM distribution with independent sources:
- 2019 ACS 5-Year Estimates, B14001: School Enrollment By Level Of School For The Population 3 Years And Over
- SANDAG Schools (CA Department of Education, IPEDS)

### SANPOPSIM

PUMS data dictionary:
https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2016-2020.pdf

SCHG = Grade level attending

In [5]:
# Summarize SCHG by count, median, min, max at the REGION level
grade_summary = synthetic_persons.groupby('SCHG')['AGEP'].agg(['count', 'median', 'min', 'max']).add_prefix('age_')
grade_summary = grade_summary.reset_index()
grade_summary.to_csv('C:/Users/dle/Downloads/school_grade_summary.csv', index=False)
grade_summary

,SCHG,age_count,age_median,age_min,age_max
0,1.0,51295,4.0,3,5
1,2.0,52363,5.0,4,7
2,3.0,47345,6.0,5,9
3,4.0,44497,7.0,5,10
4,5.0,36253,8.0,6,11
5,6.0,45415,9.0,7,12
6,7.0,37914,10.0,8,13
7,8.0,40715,11.0,9,37
8,9.0,42753,12.0,10,20
9,10.0,45103,13.0,11,50


In [6]:
# Check distribution of ages by school grades
age_grade_summary = synthetic_persons.groupby(['SCHG', 'AGEP'])['AGEP'].agg(['count'])
age_grade_summary = age_grade_summary.reset_index()
age_grade_summary.to_csv('C:/Users/dle/Downloads/school_age_grade_summary.csv', index=False)
age_grade_summary

,SCHG,AGEP,count
0,1.0,3,17972
1,1.0,4,23325
2,1.0,5,9998
3,2.0,4,2294
4,2.0,5,32839
...,...,...,...
228,16.0,75,83
229,16.0,79,95
230,16.0,80,120
231,16.0,81,47


> **Finding: odd distribution of age in grades 6-12 (SCHG 8-14) - see max ages in [grade_summary].**
>
> For example, in SCHG 10 (Grade 8), the median age is 13, but there are 243 50-year-olds also in this grade.

|SCHG | AGEP | count|
|---|----|----|
|10 | 11 | 143|
|10 | 12 | 2585|
|10 | 13 | 27982|
|10 | 14 | 12883|
|10 | 15 | 960|
|10 | 16 | 187|
|10 | 17 | 120|
|10 | 50 | 243|

In [7]:
# Summarize SCHG by count, median, min, max at the TRACT level
grade_summary_tract = synthetic_persons.groupby(['tract', 'SCHG'])['AGEP'].agg(['count', 'median', 'min', 'max']).add_prefix('age_')
grade_summary_tract = grade_summary_tract.reset_index()
grade_summary_tract.to_csv('C:/Users/dle/Downloads/grade_summary_tract.csv', index=False)
grade_summary_tract

,tract,SCHG,age_count,age_median,age_min,age_max
0,100.0,1.0,34,4.0,4,4
1,100.0,2.0,13,5.0,5,5
2,100.0,3.0,73,7.0,6,7
3,100.0,5.0,60,8.0,8,8
4,100.0,6.0,3,8.0,8,8
...,...,...,...,...,...,...
9627,22100.0,12.0,82,15.0,15,16
9628,22100.0,13.0,160,16.0,15,17
9629,22100.0,14.0,368,18.0,15,18
9630,22100.0,15.0,830,20.0,18,73


### ACS Data

In [8]:
# B14001: School Enrollment By Level Of School For The Population 3 Years And Over

# REGION LEVEL
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=ddamwsql16.sandag.org;'
                      'Database=census;'
                      'Trusted_Connection=yes;')

query = '''SELECT subject_table, line_desc, SUM(estimate) as estimate
            FROM [census].[acs].[vw_summary_file]
            WHERE yr = 2019
              AND release_type = '5Y'
              AND county = 073
              AND summary_level = 140
              AND subject_table = 'B14001'
            GROUP BY subject_table, line_desc, line_number
            ORDER BY line_number'''

acs_data = pd.read_sql_query(query,conn)
acs_data

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_table,line_desc,estimate
0,B14001,Total:,3190957.0
1,B14001,Enrolled in school:,868720.0
2,B14001,"Enrolled in nursery school, preschool",51962.0
3,B14001,Enrolled in kindergarten,42001.0
4,B14001,Enrolled in grade 1 to grade 4,149896.0
5,B14001,Enrolled in grade 5 to grade 8,160233.0
6,B14001,Enrolled in grade 9 to grade 12,166153.0
7,B14001,"Enrolled in college, undergraduate years",245624.0
8,B14001,Graduate or professional school,52851.0
9,B14001,Not enrolled in school,2322237.0


In [9]:
# CENSUS TRACT LEVEL

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=ddamwsql16.sandag.org;'
                      'Database=census;'
                      'Trusted_Connection=yes;')

query = '''SELECT subject_table, CAST(tract as INT) as trct, line_desc, SUM(estimate) as estimate
            FROM [census].[acs].[vw_summary_file]
            WHERE yr = 2019
              AND release_type = '5Y'
              AND county = 073
              AND summary_level = 140
              AND subject_table = 'B14001'
              AND line_desc NOT IN ('Total:', 'Enrolled in school:', 'Not enrolled in school')
            GROUP BY tract, subject_table, table_description, line_desc, line_number
            ORDER BY tract, line_number'''

acs_data = pd.read_sql_query(query,conn)
acs_data

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,subject_table,trct,line_desc,estimate
0,B14001,100,"Enrolled in nursery school, preschool",48.0
1,B14001,100,Enrolled in kindergarten,21.0
2,B14001,100,Enrolled in grade 1 to grade 4,172.0
3,B14001,100,Enrolled in grade 5 to grade 8,56.0
4,B14001,100,Enrolled in grade 9 to grade 12,125.0
...,...,...,...,...
4391,B14001,990100,Enrolled in grade 1 to grade 4,0.0
4392,B14001,990100,Enrolled in grade 5 to grade 8,0.0
4393,B14001,990100,Enrolled in grade 9 to grade 12,0.0
4394,B14001,990100,"Enrolled in college, undergraduate years",0.0


In [10]:
acs_data.to_csv('C:/Users/dle/Downloads/acs_summary_tract.csv', index=False)

### Department of Education Data

In [11]:
# 2019 public/private school enrollment

query = ''' SELECT * 
			  FROM [socioec_data].[ca_doe].[public_schools_enrollment]
			  WHERE yr = 2019'''

pub_sch = pd.read_sql_query(query,conn)

query = ''' SELECT * 
			  FROM [socioec_data].[ca_doe].[private_schools_enrollment]
			  WHERE yr = 2019'''

priv_sch = pd.read_sql_query(query,conn)

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
pub_sch_sum = pd.melt(pub_sch, id_vars='yr', 
                      value_vars=['kdgn', 'gr_1', 'gr_2', 'gr_3', 'gr_4', 'gr_5', 'gr_6',
                                  'gr_7', 'gr_8', 'gr_9', 'gr_10', 'gr_11', 'gr_12'],
                      var_name='grade', value_name='total')
pub_sch_sum = pub_sch_sum.groupby('grade')['total'].agg(['sum']).add_prefix('pub_').reset_index()

priv_sch_sum = pd.melt(priv_sch, id_vars='yr', 
                      value_vars=['kdgn', 'gr_1', 'gr_2', 'gr_3', 'gr_4', 'gr_5', 'gr_6',
                                  'gr_7', 'gr_8', 'gr_9', 'gr_10', 'gr_11', 'gr_12'],
                      var_name='grade', value_name='total')

priv_sch_sum = priv_sch_sum.groupby('grade')['total'].agg(['sum']).add_prefix('priv_').reset_index()

sch_sum = pd.merge(pub_sch_sum, priv_sch_sum, on='grade')
sch_sum['tot_sum'] = sch_sum['pub_sum'] + sch_sum['priv_sum']
sch_sum

,grade,pub_sum,priv_sum,tot_sum
0,gr_1,37304,2411.0,39715.0
1,gr_10,40092,3346.0,43438.0
2,gr_11,38842,3255.0,42097.0
3,gr_12,42040,3299.0,45339.0
4,gr_2,37222,2412.0,39634.0
5,gr_3,37286,2419.0,39705.0
6,gr_4,36275,2461.0,38736.0
7,gr_5,37256,2588.0,39844.0
8,gr_6,38401,2873.0,41274.0
9,gr_7,39455,2832.0,42287.0


## QC Test 12: Class of Worker vs. EDD

Compare 'COW' data in synthetic persons output with EDD CES (Current Employment Statistics) data for San Diego-Carlsbad MSA

In [41]:
# Connecting to EDD data via SODA API
# https://data.edd.ca.gov/Industry-Information-/Current-Employment-Statistics-CES-/r4zm-kdcg/data

# Filter using San Diego-Carlsbad MSA, not seasonally adjusted, and 12/1/2019

data_url = 'data.edd.ca.gov'
data_set = 'r4zm-kdcg'
client = Socrata(data_url, None)
results = client.get(
    data_set,
    where="area_name='San Diego-Carlsbad MSA' AND seasonally_adjusted='N' AND date='2019-12-01T00:00:00.000'"
    )
results_df = pd.DataFrame.from_records(results).sort_values(by=['series_code'])
results_df

,area_type,area_name,year,month,date,series_code,industry_title,seasonally_adjusted,current_employment
22,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,0,Total Nonfarm,N,1524400
42,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,1000000,Total Wage and Salary,N,1533600
19,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,10000000,Mining and Logging,N,300
55,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,11000000,Total Farm,N,9200
69,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,15000000,"Natural Resources, Mining and Constructi",N,84800
...,...,...,...,...,...,...,...,...,...
53,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,90932000,Local Government Excluding Education,N,70000
1,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,90932994,Special Districts plus Indian Tribes,N,30400
86,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,90939012,County,N,20300
41,Metropolitan Area,San Diego-Carlsbad MSA,2019,December,2019-12-01T00:00:00.000,90939022,City,N,19300


In [42]:
# Categorize EDD CES industry_title for comparison
industry_list = ['Total Private', 'Local Government', 'State Government', 'Federal Government']

edd_df = results_df[results_df['industry_title'].isin(industry_list)]
edd_df = edd_df[['industry_title', 'current_employment']]
edd_df

,industry_title,current_employment
99,Total Private,1273100
60,Federal Government,48200
0,State Government,50300
52,Local Government,152800


In [43]:
# Categorize popsim Class of Worker for comparison with EDD CES data

cow_dict = {
    1 : 'Total Private',
    2 : 'Total Private',
    3 : 'Local Government',
    4 : 'State Government',
    5 : 'Federal Government'
}

cow_df = synthetic_persons.copy()
cow_df['industry_title'] = synthetic_persons['COW'].map(cow_dict)
cow_df = cow_df[cow_df['industry_title'].notnull()]
cow_df


,tract,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,isWorker,work_status,industry_title
0,18100.0,245,1,1,52,2,1.0,1.0,40.0,NaN,1,2,4.0,1,FT,Total Private
1,18100.0,245,1,2,39,1,3.0,1.0,40.0,NaN,1,2,4.0,0,FT,Total Private
3,18100.0,245,2,1,25,2,4.0,5.0,50.0,15.0,6,2,1.0,1,FT,Federal Government
6,18100.0,245,3,1,64,2,1.0,4.0,60.0,NaN,1,1,4.0,1,FT,State Government
10,18100.0,245,4,2,43,1,1.0,1.0,60.0,NaN,1,1,4.0,1,FT,Total Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356956,NaN,24280,1262824,1,27,1,6.0,1.0,40.0,NaN,1,2,4.0,0,FT,Total Private
3356958,NaN,24280,1262826,1,33,1,6.0,1.0,40.0,NaN,1,1,2.0,0,FT,Total Private
3356961,NaN,24280,1262829,1,33,1,6.0,1.0,40.0,NaN,1,1,2.0,0,FT,Total Private
3356967,NaN,24280,1262835,1,38,1,6.0,1.0,NaN,NaN,1,2,4.0,0,NaN,Total Private


In [44]:
# Summarize COW data by industry category

cow_summary = cow_df.groupby(['industry_title'])['isWorker'].agg(['sum'])
cow_summary = cow_summary.reset_index()
cow_summary

,industry_title,sum
0,Federal Government,131785
1,Local Government,88214
2,State Government,57490
3,Total Private,1070074


In [45]:
# Compare COW data vs EDD data

ind_summary = pd.merge(cow_summary, edd_df, on='industry_title')
ind_summary = ind_summary.astype({'current_employment':'int'})
ind_summary['diff'] = ind_summary['sum'] - ind_summary['current_employment']
ind_summary = ind_summary.rename(columns = { 'sum' : 'SANPOPSIM', 'current_employment' : 'EDD'})
ind_summary

,industry_title,SANPOPSIM,EDD,diff
0,Federal Government,131785,48200,83585
1,Local Government,88214,152800,-64586
2,State Government,57490,50300,7190
3,Total Private,1070074,1273100,-203026


In [46]:
ind_summary.to_csv('C:/Users/dle/Downloads/worker_cow_edd.csv', index=False)

> **Finding: Inconclusive** - these differences are abnormally large.

In [18]:
# Checking old synthetic population for industry level data?
old_synthetic_persons = pd.read_csv(r'C:\Users\dle\OneDrive - San Diego Association of Governments\Projects\2022\2022-58 2019 Base Year Forecast Output QC\data\MGRA13 Updated Data\persons_2019_01.csv')
old_synthetic_persons.head()

,hhid,perid,household_serial_no,pnum,age,sex,miltary,pemploy,pstudent,ptype,educ,grade,occen5,occsoc5,indcen,weeks,hours,rac1p,hisp,version
0,1,1,0,1,77,1,0,3,3,5,13,0,0,00-0000,0,5,0,1,1,0
1,2,2,0,1,18,2,0,2,3,2,9,0,0,51-1011,0,5,0,8,2,0
2,2,3,0,2,55,2,0,1,3,1,9,0,0,41-1011,0,1,35,1,2,0
3,2,4,0,3,63,1,0,1,3,1,13,0,0,11-1021,0,1,35,1,2,0
4,3,5,0,1,69,1,0,3,3,5,9,0,0,41-1011,0,5,0,1,1,0


In [29]:
# Checking for unique values in occsoc5 and indcen fields
print(old_synthetic_persons.occsoc5.unique())
print(old_synthetic_persons.indcen.unique())

# Unusual that these are the only unique values

['00-0000' '51-1011' '41-1011' '11-1021' '31-1010' '45-1010' '55-1010']
[   0 9770]


<a id='workers_ratio'></a>
## QC Test 22: Ratio of workers

- Ratio of total workers to total employment
- Ratio of total part-time workers to total employment

In [30]:
# Get total employment from 2019 base year forecast
forecast_mgra = pd.read_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_15_2019_CSV_outputs/mgra_2019_CSV_Data_ind_QA.csv')
forecast_mgra.head()


,mgra,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,i1,...,midpriceroom,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres
0,1,176,110,66,0,172,107,65,0,29,...,0,0,0.0,87,1,27,0.0,0.0,0.0,0.0
1,2,56,0,56,0,52,0,52,0,9,...,0,0,0.0,103,1,27,0.0,0.0,0.0,0.0
2,3,200,23,177,0,195,23,172,0,14,...,0,0,0.0,142,1,27,0.0,0.0,0.0,0.0
3,4,4,4,0,0,2,2,0,0,2,...,0,0,0.0,221,1,27,0.0,0.0,0.0,0.0
4,5,43,43,0,0,39,39,0,0,1,...,0,0,0.0,221,1,27,0.0,0.0,0.0,0.0


In [36]:
forecast_emp_sum = forecast_mgra['emp_total'].sum()
forecast_emp_sum

1714155

In [37]:
# Classify Part-Time/Full-Time workers based on WKHP (hours worked)
workers = synthetic_persons
workers.loc[(workers.WKHP < 40) & (workers.WKHP > 0), 'work_status'] = 'PT'
workers.loc[(workers.WKHP >= 40), 'work_status'] = 'FT'
workers

,tract,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,isWorker,work_status
0,18100.0,245,1,1,52,2,1.0,1.0,40.0,NaN,1,2,4.0,1,FT
1,18100.0,245,1,2,39,1,3.0,1.0,40.0,NaN,1,2,4.0,0,FT
2,18100.0,245,1,3,79,2,6.0,NaN,NaN,NaN,1,2,4.0,0,NaN
3,18100.0,245,2,1,25,2,4.0,5.0,50.0,15.0,6,2,1.0,1,FT
4,18100.0,245,2,2,24,1,1.0,6.0,20.0,15.0,1,1,2.0,1,PT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356964,NaN,24280,1262832,1,55,2,6.0,NaN,NaN,NaN,3,1,4.0,0,NaN
3356965,NaN,24280,1262833,1,94,2,6.0,NaN,NaN,NaN,2,1,4.0,0,NaN
3356966,NaN,24280,1262834,1,68,1,6.0,NaN,NaN,NaN,1,1,2.0,0,NaN
3356967,NaN,24280,1262835,1,38,1,6.0,1.0,NaN,NaN,1,2,4.0,0,NaN


In [40]:

workers_sum = workers.groupby(['work_status'])['isWorker'].agg(['count'])
worker_ratio = workers_sum.T # Transpose table

# Ratio of total workers to total employment
worker_ratio['TotalWorker_ratio'] = (worker_ratio['FT'] + worker_ratio['PT'])/forecast_emp_sum

# Ratio of PT to total workers
worker_ratio['PT_ratio'] = worker_ratio['PT']/(worker_ratio['PT'] + worker_ratio['FT'])

worker_ratio

work_status,FT,PT,TotalWorker_ratio,PT_ratio
count,1157548,530220,0.984606,0.314155
